# 학습에 도움이 되는 테크닉

## 1. Learning Rate Scheduler

학습 중 학습률(learning rate, LR)을 조정

 ![image.png](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbeNGrz%2FbtqQzRMQAQx%2FE5HgPj8oe4AugHgHxS7RlK%2Fimg.png)

### optim.lr_scheduler
 - [doc] (https://pytorch.org/docs/stable/optim.html)

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD, lr_scheduler

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [ ]:
lr = 0.01
step_size = 30
gamma = 0.1

model = Net()

# 옵티마이저와 스케쥴러를 설정합니다
optimizer = SGD(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma) # step_size마다 학습률에 gamma를 곱합니다

lrs = []
for i in range(100):
    lrs.append(scheduler.get_last_lr()[0])
    optimizer.step()
    scheduler.step() # 스케쥴러를 사용하기 위해서 필수적으로 들어가야 합니다

plt.plot(range(100), lrs)
plt.show()

## 2. Early Stopping

Validation 데이터셋을 두고, validation 데이터셋의 예측 결과를 통해 학습 중단 시기를 결정하는 기법

![image.png](https://miro.medium.com/max/1400/1*06sTlOC3AYeZAjzUDwbaMw@2x.jpeg)

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import FashionMNIST

In [ ]:
# 학습 데이터셋과 테스트 데이터셋을 다운로드받습니다
train_dataset = FashionMNIST(
    root="./data/", train=True, download=True, transform=transforms.ToTensor()
)
valid_dataset = FashionMNIST(
    root="./data/", train=False, download=False, transform=transforms.ToTensor()
)

# 데이터로더를 생성합니다
batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# 학습을 수행할 디바이스를 설정합니다
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# 모델을 디바이스에 올립니다
model.to(device)

In [ ]:
count = 0
best_acc = 0
patience = 3

for _ in range(100000):
    correct = 0

    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    model.eval()
    valid_acc = 0
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.max(dim=1)[1]
            correct += (pred == target).sum().item()
        valid_acc = 100 * correct / len(valid_dataset) # 지정한 성능 매트릭으로 평가합니다

    if valid_acc > best_acc: # 더 좋은 성능이 나온다면
        best_acc = valid_acc # 최고 성능 기록을 갱신합니다
        count = 0 # 카운트를 초기화합니다
        torch.save(model.state_dict(), './model.pt') # 모델을 저장합니다
    else: # 성능이 오르지 않는다면
        count += 1 # 카운트를 추가합니다
        if count >= patience: # 참기로 한 횟수(patience)만큼 성능이 오르지 않았다면
            break # 학습을 멈춥니다

## 3. Dropout : `nn.Dropout2d`
학습 중 뉴런을 일시적으로 제거(drop)하는 기법
- Dropout rate가 0.5인 경우, 4개의 뉴런은 각각 0.5의 확률로 이번 학습에서 제거될지 말지를 결정
  - [내용 참고](https://heytech.tistory.com/127)

![imgae.png](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2Fm5YtD%2FbtrbwgszFXZ%2FxdUhkeyK2lmrQXYy7HioL0%2Fimg.png)

- [doc] (https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html)

- [N, C, H, W] = [N, 1, 32, 32] 사이즈의 입력이 들어갈 예정
- 예측 클래스는 8개
- 3개의 fully connected layer로 구성됨 (fc1, fc2, fc3)
    - 활성함수는 ReLU를 사용하며, 마지막 레이어에는 활성함수가 없음
- 첫 dropout은 input layer에 적용함
    - Dropout은 25% 확률로 실행함
- 두번째 dropout은 fc2에 적용함
    - Dropout은 20% 확률로 실행함

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32 * 32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 8)
        self.dropout1 = nn.Dropout(p=0.25)
        self.dropout2 = nn.Dropout(p=0.2)

    def forward(self, x):
        x = x.view(-1, 32 * 32)
        x = self.dropout1(x)
        x = F.relu(self.fc1(x))
        x = self.dropout2(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32 * 32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 8)

    def forward(self, x):
        x = x.view(-1, 32 * 32)
        x = F.dropout(x, p=0.25)
        x = F.relu(self.fc1(x))
        x = F.dropout(F.relu(self.fc2(x)), p=0.2)
        x = self.fc3(x)
        return x

## 4. Batch Normalization

- 학습 과정에서 각 배치 단위 별로 데이터가 다양한 분포를 가지더라도 각 배치별로 평균과 분산을 이용해 정규화하는 것을 뜻함
- batch 단위나 layer에 따라서 입력 값의 분포가 모두 다르지만 정규화를 통하여 분포를 zero mean gaussian 형태로 만듦
    - [내용 참고](https://gaussian37.github.io/dl-concept-batchnorm/)

![image.png](https://gaussian37.github.io/assets/img/dl/concept/batchnorm/4.png)

- [doc] (https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32 * 32, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.fc3 = nn.Linear(128, 4)

    def forward(self, x):
        x = x.view(-1, 32 * 32)
        x = self.fc1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = F.relu(x)
        return self.fc3(x)

In [ ]:
model = Net()
model(torch.rand(2, 32, 32))